### Download 20 movies for each genre 

#### Genre list

In [50]:
### Download the genre list from TMDB

import requests

url = "https://api.themoviedb.org/3/genre/movie/list?api_key=783918f9b8efd0f2ff0792d7b7de9fa2&language=en-US"

payload = "{}"
response = requests.request("GET", url, data=payload)

# reformat the result
movie_genres = response.json()
movie_genres = movie_genres["genres"]

# now the genres are in a dictionary
for i in range(len(movie_genres)):
    print "id: ", movie_genres[i]["id"], "; Genre name: ", movie_genres[i]["name"]

id:  28 ; Genre name:  Action
id:  12 ; Genre name:  Adventure
id:  16 ; Genre name:  Animation
id:  35 ; Genre name:  Comedy
id:  80 ; Genre name:  Crime
id:  99 ; Genre name:  Documentary
id:  18 ; Genre name:  Drama
id:  10751 ; Genre name:  Family
id:  14 ; Genre name:  Fantasy
id:  36 ; Genre name:  History
id:  27 ; Genre name:  Horror
id:  10402 ; Genre name:  Music
id:  9648 ; Genre name:  Mystery
id:  10749 ; Genre name:  Romance
id:  878 ; Genre name:  Science Fiction
id:  10770 ; Genre name:  TV Movie
id:  53 ; Genre name:  Thriller
id:  10752 ; Genre name:  War
id:  37 ; Genre name:  Western


In [3]:
###Export genre list 
import csv
with open("genre_list.csv", "wb") as file:
    csv_file = csv.writer(file) 
    
    # Add column names
    csv_file.writerow(["id", "GenreName"])
    
    for item in movie_genres:
            csv_file.writerow([item['id'], item['name']])

#### 20 movies for each genre

In [32]:
### function to download first 20 movies for a genre
# input: genre id
# output: json file

def get_movie_genre(genre_id):
    
    # construct the url
    url_1 = "https://api.themoviedb.org/3/genre/"
    url_2 = "/movies?api_key=783918f9b8efd0f2ff0792d7b7de9fa2&language=en-US&include_adult=false&sort_by=created_at.asc"
    url =  url_1 + str(genre_id) + url_2
    
    # download the movies
    payload = "{}"
    response = requests.request("GET", url, data=payload)
    
    # convert to json file
    movie_list = response.json()
    movie_list = movie_list["results"]
    
    return movie_list

In [53]:
### download first 20 movies for each genre
import time

# list to store movie lists 
movie_by_genre = list()

for i in range(len(movie_genres)):
    # read genre id
    genre_id = movie_genres[i]["id"]
    
    # download movie list for the genre
    movie_list = get_movie_genre(genre_id)
    movie_by_genre.append(movie_list)
    
    time.sleep(5)

In [144]:
### replace some data that cause problem

# spot missing values in the data, change it to 'NA'
movie_by_genre[5][17]["overview"] = "NA"
movie_by_genre[5][18]["backdrop_path"] = "NA"

# replace title in French to English (remove \x)
movie_by_genre[5][18]["original_title"] = "Paris est une fxeate - Un film en 18 vagues"
movie_by_genre[5][18]["title"] = "Paris est une fxeate - Un film en 18 vagues"

In [146]:
### Save json contents as CSV
import csv
import json

# list of items in the data
features = ['poster_path', 'title', 'overview', 'popularity', 
        'original_title', 'backdrop_path','vote_count', 'video', 
        'adult', 'vote_average', 'original_language', 'id', 'genre_ids',]

# save information into a csv file
with open("movie_by_genres.csv", "wb") as file:
    csv_file = csv.writer(file) 
    
    # Add column names
    csv_file.writerow(features)
    
    # For each item in list, get attributes of movie
    for i in range(len(movie_by_genre)):
        # remove documentary because it has missing values and causes problems
        for item in movie_by_genre[i]:
            csv_file.writerow([item['poster_path'], item['title'], item['overview'].encode("utf8"), 
                               item['popularity'], item['original_title'].encode("utf8"), item['backdrop_path'], 
                               item['vote_count'], item['video'], item['adult'], item['vote_average'],
                               item['original_language'], item['id'], item['genre_ids']])

#### One-hot-encode genres 

In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter

In [12]:
# read in the saved csv file
movie_by_genre = pd.read_csv("movie_by_genres.csv")

In [13]:
# format genre ids into a list of ids
movie_by_genre['genre_ids'] = [map(int, re.sub("[\[ \] ]", "", movie_by_genre['genre_ids'][i]).split(',')) for i in range(movie_by_genre.shape[0])]

# obe-hot-encode genres
data_count_genres = movie_by_genre['genre_ids'].apply(Counter)
one_hot_encode_genres = pd.DataFrame.from_records(data_count_genres).fillna(value=int(0))
movie_by_genre = movie_by_genre.join(one_hot_encode_genres)

In [14]:
### replace genre id by genre names

# read the genre list
genre_names = pd.read_csv("genre_list.csv")
# conver the dataframe into a dictionary
genre_names = genre_names.set_index("id")
genre_name_dict = genre_names.to_dict()['GenreName']

# match genre name to the id
movie_by_genre = movie_by_genre.rename(columns = genre_name_dict)

In [15]:
# remove duplicated movies
movie_by_genre = movie_by_genre.drop_duplicates("id")

# new dimension
movie_by_genre.shape

(246, 32)

In [16]:
# save the file to a csv
movie_by_genre.to_csv("movie_by_genre_onehot.csv", index = None)